## 1. Silver Layer (Enhanced Dataset)

In [0]:
CREATE OR REPLACE TABLE uber_rides_enhanced AS
SELECT 
    *,
    HOUR(tpep_pickup_datetime) AS pickup_hour,
    DAYOFWEEK(tpep_pickup_datetime) AS pickup_day_of_week,
    CASE DAYOFWEEK(tpep_pickup_datetime)
        WHEN 1 THEN 'Sunday' WHEN 2 THEN 'Monday' WHEN 3 THEN 'Tuesday'
        WHEN 4 THEN 'Wednesday' WHEN 5 THEN 'Thursday' 
        WHEN 6 THEN 'Friday' WHEN 7 THEN 'Saturday'
    END AS pickup_day_name,
    MONTH(tpep_pickup_datetime) AS pickup_month,
    YEAR(tpep_pickup_datetime) AS pickup_year,
    CASE 
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 6 AND 11 THEN 'Morning'
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 12 AND 17 THEN 'Afternoon'
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 18 AND 22 THEN 'Evening'
        ELSE 'Night'
    END AS time_of_day,
    CASE 
        WHEN DAYOFWEEK(tpep_pickup_datetime) IN (1, 7) THEN 1 
        ELSE 0 
    END AS is_weekend,
    CASE 
        WHEN HOUR(tpep_pickup_datetime) BETWEEN 7 AND 9 
          OR HOUR(tpep_pickup_datetime) BETWEEN 17 AND 19 THEN 1
        ELSE 0 
    END AS is_rush_hour,
    ROUND((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60, 2) AS trip_duration_minutes,
    CASE 
        WHEN fare_amount > 0 THEN ROUND(tip_amount / fare_amount * 100, 2)
        ELSE 0 
    END AS tip_percentage,
    CASE 
        WHEN trip_distance > 0 THEN ROUND(total_amount / trip_distance, 2)
        ELSE 0 
    END AS cost_per_mile,
    CASE 
        WHEN (UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60 > 0 
            THEN ROUND(trip_distance / (((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 60) / 60), 2)
        ELSE 0 
    END AS speed_mph,
    CASE payment_type
        WHEN 1 THEN 'Credit Card'
        WHEN 2 THEN 'Cash'
        WHEN 3 THEN 'No Charge'
        WHEN 4 THEN 'Dispute'
        WHEN 5 THEN 'Unknown'
        WHEN 6 THEN 'Voided Trip'
        ELSE 'Other'
    END AS payment_method
FROM uber_data
WHERE trip_distance > 0 
    AND fare_amount > 0 
    AND total_amount > 0
    AND passenger_count BETWEEN 1 AND 8
    AND pickup_latitude BETWEEN 40.4 AND 41.0
    AND pickup_longitude BETWEEN -74.3 AND -73.7;

SELECT 'Enhanced dataset created successfully!' AS status,
       COUNT(*) AS clean_records
FROM uber_rides_enhanced;

status,clean_records
Enhanced dataset created successfully!,98574


In [0]:
SELECT * 
FROM uber_rides_enhanced
LIMIT 10;

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_hour,pickup_day_of_week,pickup_day_name,pickup_month,pickup_year,time_of_day,is_weekend,is_rush_hour,trip_duration_minutes,tip_percentage,cost_per_mile,speed_mph,payment_method
1,2016-03-01T00:00:00.000Z,2016-03-01T00:07:55.000Z,1,2.5,-73.97674560546875,40.765151977539055,1,N,-74.00426483154298,40.74612808227539,1,9.0,0.5,0.5,2.05,0.0,0.3,12.35,0,3,Tuesday,3,2016,Night,0,0,7.92,22.78,4.94,18.95,Credit Card
1,2016-03-01T00:00:00.000Z,2016-03-01T00:11:06.000Z,1,2.9,-73.98348236083984,40.767925262451165,1,N,-74.00594329833984,40.7331657409668,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35,0,3,Tuesday,3,2016,Night,0,0,11.1,27.73,5.29,15.68,Credit Card
2,2016-03-01T00:00:00.000Z,2016-03-01T00:31:06.000Z,2,19.98,-73.78202056884764,40.64480972290039,1,N,-73.97454071044923,40.6757698059082,1,54.5,0.5,0.5,8.0,0.0,0.3,63.8,0,3,Tuesday,3,2016,Night,0,0,31.1,14.68,3.19,38.55,Credit Card
2,2016-03-01T00:00:00.000Z,2016-03-01T00:00:00.000Z,3,10.78,-73.86341857910156,40.769813537597656,1,N,-73.96965026855469,40.757766723632805,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,0,3,Tuesday,3,2016,Night,0,0,0.0,12.0,3.86,0.0,Credit Card
2,2016-03-01T00:00:00.000Z,2016-03-01T00:00:00.000Z,5,30.43,-73.97174072265625,40.79218292236328,3,N,-74.17716979980467,40.69505310058594,1,98.0,0.0,0.0,0.0,15.5,0.3,113.8,0,3,Tuesday,3,2016,Night,0,0,0.0,0.0,3.74,0.0,Credit Card
2,2016-03-01T00:00:00.000Z,2016-03-01T00:00:00.000Z,5,5.92,-74.01719665527344,40.70538330078125,1,N,-73.97807312011719,40.75578689575195,1,23.5,1.0,0.5,5.06,0.0,0.3,30.36,0,3,Tuesday,3,2016,Night,0,0,0.0,21.53,5.13,0.0,Credit Card
2,2016-03-01T00:00:00.000Z,2016-03-01T00:00:00.000Z,6,5.72,-73.99458312988281,40.72784805297852,1,N,0.0,0.0,2,23.0,0.5,0.5,0.0,0.0,0.3,24.3,0,3,Tuesday,3,2016,Night,0,0,0.0,0.0,4.25,0.0,Cash
1,2016-03-01T00:00:01.000Z,2016-03-01T00:16:04.000Z,1,6.2,-73.78877258300781,40.64775848388672,1,N,-73.82920837402342,40.712345123291016,3,20.5,0.5,0.5,0.0,0.0,0.3,21.8,0,3,Tuesday,3,2016,Night,0,0,16.05,0.0,3.52,23.18,No Charge
1,2016-03-01T00:00:01.000Z,2016-03-01T00:05:00.000Z,1,0.7,-73.95822143554686,40.76464080810546,1,N,-73.9678955078125,40.76290130615234,1,5.5,0.5,0.5,2.0,0.0,0.3,8.8,0,3,Tuesday,3,2016,Night,0,0,4.98,36.36,12.57,8.43,Credit Card
2,2016-03-01T00:00:01.000Z,2016-03-01T00:24:06.000Z,3,7.18,-73.98577880859375,40.74119186401367,1,N,-73.94635009765625,40.79787826538086,1,23.5,0.5,0.5,3.2,0.0,0.3,28.0,0,3,Tuesday,3,2016,Night,0,0,24.08,13.62,3.9,17.89,Credit Card


## 2. Executive KPI Dashboard Queries

### A. Core Business Metrics

In [0]:

SELECT 
    'EXECUTIVE DASHBOARD' as dashboard_title,
    COUNT(*) as total_trips,
    ROUND(SUM(total_amount), 0) as total_revenue_dollars,
    ROUND(AVG(fare_amount), 2) as avg_fare_dollars,
    ROUND(AVG(tip_percentage), 1) as avg_tip_percentage,
    ROUND(AVG(trip_distance), 1) as avg_trip_miles,
    ROUND(AVG(trip_duration_minutes), 1) as avg_duration_minutes,
    COUNT(DISTINCT pickup_day_name) as days_analyzed,
    ROUND(SUM(total_amount) / COUNT(DISTINCT DATE(tpep_pickup_datetime)), 0) as daily_avg_revenue
FROM uber_rides_enhanced;

dashboard_title,total_trips,total_revenue_dollars,avg_fare_dollars,avg_tip_percentage,avg_trip_miles,avg_duration_minutes,days_analyzed,daily_avg_revenue
EXECUTIVE DASHBOARD,98574,1611566.0,13.21,14.0,3.1,17.0,2,805783.0


### B. Peak Performance Analysis

In [0]:
WITH hourly_stats AS (
    SELECT 
        pickup_hour,
        COUNT(*) as trips,
        ROUND(SUM(total_amount), 0) as revenue,
        ROUND(AVG(fare_amount), 2) as avg_fare,
        ROUND(AVG(tip_percentage), 1) as avg_tip_pct,
        ROUND(AVG(speed_mph), 1) as avg_speed
    FROM uber_rides_enhanced
    WHERE speed_mph BETWEEN 1 AND 80  -- Remove outliers
    GROUP BY pickup_hour
),
ranked_hours AS (
    SELECT *,
        RANK() OVER (ORDER BY revenue DESC) as revenue_rank,
        RANK() OVER (ORDER BY trips DESC) as volume_rank
    FROM hourly_stats
)
SELECT 
    pickup_hour,
    CASE 
        WHEN pickup_hour BETWEEN 7 AND 9 THEN 'Morning Rush'
        WHEN pickup_hour BETWEEN 17 AND 19 THEN 'Evening Rush'
        WHEN pickup_hour BETWEEN 22 AND 5 THEN 'Late Night'
        ELSE 'Regular Hours'
    END as time_category,
    trips,
    revenue,
    avg_fare,
    avg_tip_pct,
    avg_speed,
    revenue_rank,
    volume_rank
FROM ranked_hours
ORDER BY pickup_hour;

pickup_hour,time_category,trips,revenue,avg_fare,avg_tip_pct,avg_speed,revenue_rank,volume_rank
0,Regular Hours,6871,118748.0,13.87,14.1,17.6,8,8
1,Regular Hours,4051,68134.0,13.64,13.2,18.1,10,10
2,Regular Hours,2542,39913.0,12.82,12.5,18.0,12,12
3,Regular Hours,1820,31034.0,14.06,11.0,18.7,15,15
4,Regular Hours,1867,36926.0,16.04,11.0,21.3,13,13
5,Regular Hours,3594,66002.0,14.38,13.8,19.8,11,11
6,Regular Hours,1860,31631.0,13.56,13.4,17.6,14,14
7,Morning Rush,9415,140174.0,11.98,14.9,11.7,7,7
8,Morning Rush,11543,174069.0,12.16,15.2,9.9,1,1
9,Morning Rush,10566,164861.0,12.57,15.0,9.6,4,2


### C. Geographic Revenue Hotspots

In [0]:
WITH location_analysis AS (
    SELECT 
        ROUND(pickup_latitude, 3) as lat,
        ROUND(pickup_longitude, 3) as lng,
        COUNT(*) as trip_count,
        ROUND(SUM(total_amount), 0) as total_revenue,
        ROUND(AVG(fare_amount), 2) as avg_fare,
        ROUND(AVG(tip_percentage), 1) as avg_tip_pct,
        ROUND(AVG(trip_distance), 1) as avg_distance
    FROM uber_rides_enhanced
    GROUP BY ROUND(pickup_latitude, 3), ROUND(pickup_longitude, 3)
    HAVING COUNT(*) >= 20  -- Focus on significant locations
)
SELECT 
    'TOP PICKUP ZONES' as analysis_type,
    lat,
    lng,
    trip_count,
    total_revenue,
    avg_fare,
    avg_tip_pct,
    avg_distance,
    RANK() OVER (ORDER BY total_revenue DESC) as revenue_rank
FROM location_analysis
ORDER BY total_revenue DESC
LIMIT 15;

analysis_type,lat,lng,trip_count,total_revenue,avg_fare,avg_tip_pct,avg_distance,revenue_rank
TOP PICKUP ZONES,40.645,-73.782,523,28166.0,45.08,11.3,15.8,1
TOP PICKUP ZONES,40.774,-73.871,449,18741.0,32.47,13.4,9.7,2
TOP PICKUP ZONES,40.645,-73.777,363,18465.0,42.9,11.5,15.4,3
TOP PICKUP ZONES,40.774,-73.873,372,16411.0,33.62,15.8,9.8,4
TOP PICKUP ZONES,40.769,-73.863,325,15368.0,35.77,17.2,10.4,5
TOP PICKUP ZONES,40.77,-73.863,283,13901.0,36.88,18.8,10.8,6
TOP PICKUP ZONES,40.77,-73.864,283,13600.0,36.63,17.4,10.6,7
TOP PICKUP ZONES,40.751,-73.994,698,9987.0,11.9,12.8,2.1,8
TOP PICKUP ZONES,40.647,-73.79,172,9235.0,44.74,11.4,15.5,9
TOP PICKUP ZONES,40.75,-73.991,527,7238.0,11.18,14.2,2.1,10


## 2. Customer Behavior Analysis

### A. Payment & Tipping Insights


In [0]:
SELECT 
    payment_method,
    COUNT(*) as trip_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) as market_share_pct,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(tip_amount), 2) as avg_tip_dollars,
    ROUND(AVG(tip_percentage), 1) as avg_tip_pct,
    ROUND(SUM(total_amount), 0) as total_revenue,
    CASE 
        WHEN AVG(tip_percentage) >= 20 THEN 'High Tippers'
        WHEN AVG(tip_percentage) >= 15 THEN 'Good Tippers'
        WHEN AVG(tip_percentage) >= 10 THEN 'Average Tippers'
        ELSE 'Low Tippers'
    END as tipping_category
FROM uber_rides_enhanced
GROUP BY payment_method
ORDER BY trip_count DESC;

payment_method,trip_count,market_share_pct,avg_fare,avg_tip_dollars,avg_tip_pct,total_revenue,tipping_category
Credit Card,65775,66.7,13.83,2.8,21.0,1181565.0,High Tippers
Cash,32655,33.1,11.96,0.0,0.0,427639.0,Low Tippers
No Charge,107,0.1,13.79,0.11,0.2,1626.0,Low Tippers
Dispute,37,0.0,18.08,0.0,0.0,737.0,Low Tippers


### B. Distance-Based Pricing Analysis

In [0]:
WITH distance_buckets AS (
    SELECT 
        CASE 
            WHEN trip_distance <= 1 THEN '0-1 miles (Short)'
            WHEN trip_distance <= 3 THEN '1-3 miles (Medium)'
            WHEN trip_distance <= 5 THEN '3-5 miles (Long)'
            WHEN trip_distance <= 10 THEN '5-10 miles (Very Long)'
            ELSE '10+ miles (Airport/Long Distance)'
        END as distance_category,
        trip_distance,
        fare_amount,
        total_amount,
        tip_percentage,
        trip_duration_minutes,
        speed_mph
    FROM uber_rides_enhanced
    WHERE trip_distance > 0 AND trip_distance <= 25  -- Remove extreme outliers
)
SELECT 
    distance_category,
    COUNT(*) as trip_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) as trip_percentage,
    ROUND(AVG(trip_distance), 1) as avg_distance,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(total_amount), 2) as avg_total,
    ROUND(AVG(tip_percentage), 1) as avg_tip_pct,
    ROUND(AVG(trip_duration_minutes), 0) as avg_duration_min,
    ROUND(AVG(fare_amount / trip_distance), 2) as fare_per_mile,
    ROUND(SUM(total_amount), 0) as total_revenue
FROM distance_buckets
GROUP BY distance_category
ORDER BY 
    CASE 
        WHEN distance_category LIKE '%0-1%' THEN 1
        WHEN distance_category LIKE '%1-3%' THEN 2
        WHEN distance_category LIKE '%3-5%' THEN 3
        WHEN distance_category LIKE '%5-10%' THEN 4
        ELSE 5
    END;

distance_category,trip_count,trip_percentage,avg_distance,avg_fare,avg_total,avg_tip_pct,avg_duration_min,fare_per_mile,total_revenue
0-1 miles (Short),25055,25.4,0.7,5.76,7.44,14.0,8.0,11.81,186484.0
1-3 miles (Medium),47191,47.9,1.8,9.98,12.26,14.0,15.0,5.87,578339.0
3-5 miles (Long),10901,11.1,3.8,16.34,19.57,13.4,21.0,4.29,213279.0
5-10 miles (Very Long),9237,9.4,7.1,24.56,30.15,13.9,29.0,3.49,278484.0
10+ miles (Airport/Long Distance),6077,6.2,14.7,44.62,56.23,15.3,41.0,3.08,341717.0
